In [1]:
import pandas as pd
import numpy as np
import locale
import seaborn as sns
import unicodedata as unicodedata
sns.set()

In [2]:
df = pd.read_csv('df_arreglado.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45376 non-null  float64
 1   id                 45376 non-null  float64
 2   original_language  45365 non-null  object 
 3   overview           44435 non-null  object 
 4   popularity         45376 non-null  float64
 5   release_date       45376 non-null  object 
 6   revenue            45376 non-null  float64
 7   runtime            45130 non-null  float64
 8   status             45296 non-null  object 
 9   tagline            20398 non-null  object 
 10  title              45376 non-null  object 
 11  vote_average       45376 non-null  float64
 12  release_year       45376 non-null  float64
 13  return             45376 non-null  float64
 14  id_collection      4488 non-null   float64
 15  name_collection    4488 non-null   object 
 16  id_genres          429

In [4]:
def peliculas_mes(mes):
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes= fechas[fechas.dt.month_name(locale = 'es')==mes.capitalize()] # anda en API local pero no en render
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

'en_US.UTF-8'

In [8]:
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [5]:
def peliculas_mes(mes):
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes = fechas[fechas.dt.strftime('%B').str.capitalize() == mes.capitalize()] # se realizo la siguiente transformacion
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

In [7]:
def peliculas_mes(mes: str) -> dict:
    # Set the locale to Spanish
    #locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

    months_translated= {
    'enero': 'January',
    'febrero': 'February',
    'marzo': 'March',
    'abril': 'April',
    'mayo': 'May',
    'junio': 'June',
    'julio': 'July',
    'agosto': 'August',
    'septiembre': 'September',
    'octubre': 'October',
    'noviembre': 'November',
    'diciembre': 'December'}  
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes = fechas[fechas.dt.strftime('%B').str.capitalize() == months_translated[str(mes).lower()]]
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}   # y por ultimo la mejora para que lo lea render

In [9]:
peliculas_mes('enero')

{'mes': 'enero', 'cantidad': 5912}

In [10]:
def peliculas_dia(dia):
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_dia= fechas[fechas.dt.day_name(locale = 'es')==dia.capitalize()] # idem con dias
    respuesta = n_dia.shape[0]
    return {'dia':dia, 'cantidad':respuesta}

In [11]:
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [12]:
import locale

def peliculas_dia(dia):

    dias_permitidos = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']

    if dia.lower() not in dias_permitidos:
        return {'error': 'El día proporcionado no es válido.'}

    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

    fechas = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

    n_dia = fechas[fechas.dt.day_name(locale='es') == dia.capitalize()]

    cantidad = n_dia.shape[0]

    return {'dia': dia, 'cantidad': cantidad}

In [13]:
def peliculas_dia(dia: str) -> dict:

    day_translated= {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sabado': 'Saturday',
    'domingo': 'Sunday'}   
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_dia = fechas[fechas.dt.strftime('%A').str.capitalize() == day_translated[str(dia).lower()]]
    respuesta = n_dia.shape[0]
    return {'mes':dia, 'cantidad':respuesta} # anda en el render

In [14]:
peliculas_dia('lunes')

{'mes': 'lunes', 'cantidad': 3503}

In [15]:
def franquicia(franquicia):
    f_bajo= franquicia.lower()
    fran= df[['name_collection','budget','revenue',]].dropna(subset=['name_collection'])
    fran= fran[fran['name_collection'].map(str.lower).apply(lambda x: f_bajo in x)]
    cantidad = fran.shape[0]
    gananciat= (fran['revenue']- fran['budget']).sum()
    gananciap= (fran['revenue']- fran['budget']).mean()
    return {'franquicia': franquicia, 'cantidad': cantidad, 'ganancia_total':gananciat, 'ganancia_promedio': gananciap} # anduvo a la primera

In [16]:
franquicia('Toy Story Collection')

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1617890605.0,
 'ganancia_promedio': 539296868.3333334}

In [17]:
def peliculas_pais(pais):
    m1 = df[['title', 'name_countrie']]
    if isinstance(pais,str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        ganancias = m1['title'][m1['name_countrie'].str.contains(pais)==True]
        respuesta = ganancias.shape[0]
    return {'pais': pais, 'cantidad': respuesta} # convine este

In [18]:
def peliculas_pais(pais: str) -> dict:
    if isinstance(pais, str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        cantidad = df['name_countrie'].apply(lambda x: str(x).lower()).map(str.lower).apply(lambda x: pais in x).sum()
        return {'pais': pais, 'cantidad': cantidad} # con este

In [19]:
def peliculas_pais(pais):
    m1 = df[['title', 'name_countrie']]
    if isinstance(pais, str):
        pais = pais.lower()
        pais = unicodedata.normalize('NFKD', pais).encode('ascii', 'ignore').decode('utf-8','ignore')
        cantidad = m1['title'][m1['name_countrie'].str.contains(pais)==True]
        cantidad = df['name_countrie'].apply(lambda x: str(x).lower()).map(str.lower).apply(lambda x: pais in x)
        respuesta = cantidad.shape[0]
    return {'pais': pais, 'cantidad': respuesta} # y salio al fin este 

In [20]:
peliculas_pais ('United States of America')

{'pais': 'united states of america', 'cantidad': 45464}

In [21]:
def productoras(productora):
    prod = df[['name_production', 'budget', 'revenue']].dropna()
    prod ['name_production'] = prod['name_production'].map(str.lower)
    cantidad = prod.shape[0]
    gtotal= (prod['revenue'] - prod['budget']).sum()
    return {'productora':productora, 'ganancia_total': gtotal, 'cantidad': cantidad } # a la primera anduvo

In [22]:
productoras('Warner Bros.')

{'productora': 'Warner Bros.',
 'ganancia_total': 257223845008.0,
 'cantidad': 33522}

In [24]:
def retorno(pelicula):
    peli = df[['title','budget', 'revenue', 'return', 'release_year']].dropna()
    peli['title'] = peli['title'].map(str.lower)
    peli = peli[peli['title']==str(pelicula).lower()]
    inver = peli['budget'].loc[0]
    gan = peli['revenue'].loc[0]
    ret = peli['return'].loc[0]
    an = peli['release_year'].loc[0]
    return {'pelicula': pelicula, 'inversion': inver, 'ganancia': gan, 'retorno': ret, 'anio': an } # este no lo leia por lo que se le hizo mejoras

In [23]:
def retorno(pelicula: str) -> dict:
    pelicula_df = df.loc[df['title'] == pelicula.title()]
    inversion = pelicula_df['budget'].iloc[0].item()
    ganancia = pelicula_df['revenue'].iloc[0].item()
    retorno= pelicula_df['return'].iloc[0].item()
    anio = pelicula_df['release_year'].iloc[0].item()
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio } # arreglado